<a href="https://colab.research.google.com/github/mjiii25/posco-academy/blob/main/Big-Data/Battery-Project/hypothesis-test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **가설 검정**

In [ ]:
from google.colab import drive
drive.mount('/content/mydrive')

Mounted at /content/mydrive


In [162]:
import pandas as pd
import numpy as np

import pandas as pd
import scipy.stats as stats
from scipy.stats import mannwhitneyu

import matplotlib.pyplot as plt
import matplotlib

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

### **데이터 구성하기**

In [163]:
df_raw_process = pd.read_csv('/content/mydrive/MyDrive/posco-big-data/bat_process.csv', encoding = 'euc-kr')


print(df_raw_process.shape)
df_raw_process.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (97,101,102,103) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(43754, 109)


,lot_id,tray_id,dt_start,cell_id,judge,rta1_cell_no,rta1_box_col,rta1_box_row,rta1_box_dan,hta1_box_col,hta1_box_row,hta1_box_dan,rta2_box_col,rta2_box_row,rta2_box_dan,ocv1_ocv,ocv1_box_col,ocv1_box_dan,c1_curr_end,c1_voltage_avg,c1_capa,c1_ccval,c1_time_cc,c1_box_col,c1_box_dan,c1_temp_avg,dc1_curr_end,dc1_voltage_avg,dc1_capa,dc1_box_col,dc1_box_dan,dc1_temp_avg,dc1_capafit,c2_curr_end,c2_voltage_avg,c2_capa,c2_ccval,c2_time_cc,c2_box_col,c2_box_dan,c2_temp_avg,dc2_curr_end,dc2_voltage_avg,dc2_capa,dc2_box_col,dc2_box_dan,dc2_temp_avg,dc2_capafit,c3_curr_end,c3_voltage_avg,c3_capa,c3_ccval,c3_time_cv,c3_cvval,c3_time_cc,c3_box_col,c3_box_dan,c3_temp_avg,dc3_curr_end,dc3_voltage_avg,dc3_capa,dc3_box_col,dc3_box_dan,dc3_temp_avg,dc3_capafit,c4_curr_end,c4_voltage_avg,c4_capa,c4_ccval,c4_time_cv,c4_cvval,c4_time_cc,c4_box_col,c4_box_dan,c4_temp_avg,ocv2_ocv,ocv2_box_col,ocv2_box_dan,pg1_impfit,pg1_imp,pg1_voltage_start,pg1_curr_end,pg1_box_col,pg1_box_dan,pg1_temp_avg,pc1_imp,pc1_voltage_start,pc1_curr_end,pc1_box_col,pc1_box_dan,sa1_box_col,sa1_box_row,sa1_box_dan,socv1_ocv,sa2_box_col,sa2_box_row,sa2_box_dan,socv2_ocv,sa3_box_col,sa3_box_row,sa3_box_dan,socv3_ocv,ocv2_deltaocv,ocv1_deltaocv,m1_thick,m1_voltage,m1_res_ac,m1_mv,m1_voltage_an
0,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10747,양품,10,08열,33연,08단,03열,25연,03단,04열,64연,06단,3333,22열,08단,6801,3781,6451,6451,3417,22열,08단,268,6799,4108,544,22열,08단,266,539,6800,4146,625,625,332,22열,08단,265,6799,4112,544,22열,08단,266,539,135,4189,749,578,1190,171,307,22열,08단,266,6796,3674,7094,22열,08단,269,7088,136,3629,4282,4072,1091,210,2157,22열,08단,273,3764,22열,08단,2211,2114,3759,120,12열,01단,288,2086,3714,150,12열,01단,09열,42연,01단,3766.1,03열,64연,05단,3761.1,14열,37연,04단,3759.3,5,6.8,4718,37583,94,565,164
1,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10748,양품,7,08열,33연,08단,03열,25연,03단,04열,64연,06단,3329,22열,08단,6797,3783,6390,6390,3385,22열,08단,268,6800,4103,545,22열,08단,266,540,6802,4144,657,657,349,22열,08단,265,6800,4110,544,22열,08단,266,539,136,4189,768,588,1286,180,313,22열,08단,266,6799,3674,7091,22열,08단,269,7085,136,3624,4268,4064,969,204,2153,22열,08단,273,3763,22열,08단,2224,2127,3758,120,12열,01단,288,2093,3714,150,12열,01단,09열,42연,01단,3765.2,03열,64연,05단,3760.2,14열,37연,04단,3758.4,5,6.8,4718,37574,94,565,73
2,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10749,양품,19,08열,33연,08단,03열,25연,03단,04열,64연,06단,3329,22열,08단,6798,3781,6436,6436,3410,22열,08단,268,6799,4107,544,22열,08단,266,539,6801,4146,630,630,335,22열,08단,265,6799,4111,544,22열,08단,266,539,136,4189,753,579,1228,174,308,22열,08단,266,6799,3674,7079,22열,08단,269,7073,136,3630,4270,4058,1112,212,2150,22열,08단,273,3764,22열,08단,2217,2120,3759,120,12열,01단,288,2081,3715,150,12열,01단,09열,42연,01단,3765.9,03열,64연,05단,3760.8,14열,37연,04단,3759,5.1,6.9,4700,37579,91,585,-57
3,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10750,양품,24,08열,33연,08단,03열,25연,03단,04열,64연,06단,3330,22열,08단,6799,3781,6474,6474,3430,22열,08단,268,6800,4108,545,22열,08단,266,540,6801,4146,627,627,333,22열,08단,265,6799,4111,544,22열,08단,266,539,136,4190,765,583,1328,182,310,22열,08단,266,6796,3675,7106,22열,08단,269,7100,136,3630,4304,4092,1129,212,2168,22열,08단,273,3764,22열,08단,2214,2117,3759,119,12열,01단,288,2083,3714,150,12열,01단,09열,42연,01단,3766.4,03열,64연,05단,3761.3,14열,37연,04단,3759.5,5.1,6.9,4688,37583,93,606,99
4,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10751,양품,17,08열,33연,08단,03열,25연,03단,04열,64연,06단,3330,22열,08단,6799,3780,6486,6486,3436,22열,08단,268,6798,4110,544,22열,08단,266,539,6800,4147,613,613,326,22열,08단,265,6799,4112,544,22열,08단,266,539,136,4187,733,575,980,159,306,22열,08단,266,6797,3674,7101,22열,08단,269,7095,136,3619,4286,4088,879,199,2166,22열,08단,273,3762,22열,08단,2221,2124,3758,120,12열,01단,288,2083,3714,150,12열,01단,09열,42연,01단,3765.0,03열,64연,05단,3760,14열,37연,04단,3758.2,5,6.8,4696,37570,91,605,36


In [164]:
df_raw_tat = pd.read_csv('/content/mydrive/MyDrive/posco-big-data/bat_tat.csv', encoding = 'euc-kr')


print(df_raw_tat.shape)
df_raw_tat.head()

(43754, 25)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (20,21,22,23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,lot_id,tray_id,dt_start,cell_id,rta1_tat,hta1_tat,rta2_tat,ocv1_tat,c1_tat,dc1_tat,c2_tat,dc2_tat,c3_tat,dc3_tat,c4_tat,ocv2_tat,pg1_tat,pc1_tat,sa1_tat,socv1_tat,sa2_tat,socv2_tat,sa3_tat,socv3_tat,sa4_tat
0,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10747,217066,86461,5732,119,3556,404,466,404,1797,3884,3428,118,290,290,53500,5,355357,5,215150,5,207532
1,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10748,217066,86461,5732,119,3556,404,466,404,1797,3884,3428,118,290,290,53500,5,355357,5,215150,5,207532
2,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10749,217066,86461,5732,118,3556,405,466,404,1797,3884,3428,118,290,290,53500,5,355357,5,215150,5,207532
3,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10750,217066,86461,5732,118,3556,405,466,404,1797,3884,3428,118,290,290,53501,5,355357,5,215150,5,207532
4,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10751,217066,86461,5732,119,3556,404,466,404,1797,3884,3428,118,290,290,53500,5,355357,5,215150,5,207532


In [ ]:
# df_raw.loc[df_raw['judge'] == '불량', 'judge'] = 1
# df_raw.loc[df_raw['judge'] == '양품', 'judge'] = 0

In [165]:
df_raw = pd.merge(df_raw_process, df_raw_tat)

print(df_raw.shape)
df_raw.head()

(43754, 130)


,lot_id,tray_id,dt_start,cell_id,judge,rta1_cell_no,rta1_box_col,rta1_box_row,rta1_box_dan,hta1_box_col,hta1_box_row,hta1_box_dan,rta2_box_col,rta2_box_row,rta2_box_dan,ocv1_ocv,ocv1_box_col,ocv1_box_dan,c1_curr_end,c1_voltage_avg,c1_capa,c1_ccval,c1_time_cc,c1_box_col,c1_box_dan,c1_temp_avg,dc1_curr_end,dc1_voltage_avg,dc1_capa,dc1_box_col,dc1_box_dan,dc1_temp_avg,dc1_capafit,c2_curr_end,c2_voltage_avg,c2_capa,c2_ccval,c2_time_cc,c2_box_col,c2_box_dan,c2_temp_avg,dc2_curr_end,dc2_voltage_avg,dc2_capa,dc2_box_col,dc2_box_dan,dc2_temp_avg,dc2_capafit,c3_curr_end,c3_voltage_avg,c3_capa,c3_ccval,c3_time_cv,c3_cvval,c3_time_cc,c3_box_col,c3_box_dan,c3_temp_avg,dc3_curr_end,dc3_voltage_avg,dc3_capa,dc3_box_col,dc3_box_dan,dc3_temp_avg,dc3_capafit,c4_curr_end,c4_voltage_avg,c4_capa,c4_ccval,c4_time_cv,c4_cvval,c4_time_cc,c4_box_col,c4_box_dan,c4_temp_avg,ocv2_ocv,ocv2_box_col,ocv2_box_dan,pg1_impfit,pg1_imp,pg1_voltage_start,pg1_curr_end,pg1_box_col,pg1_box_dan,pg1_temp_avg,pc1_imp,pc1_voltage_start,pc1_curr_end,pc1_box_col,pc1_box_dan,sa1_box_col,sa1_box_row,sa1_box_dan,socv1_ocv,sa2_box_col,sa2_box_row,sa2_box_dan,socv2_ocv,sa3_box_col,sa3_box_row,sa3_box_dan,socv3_ocv,ocv2_deltaocv,ocv1_deltaocv,m1_thick,m1_voltage,m1_res_ac,m1_mv,m1_voltage_an,rta1_tat,hta1_tat,rta2_tat,ocv1_tat,c1_tat,dc1_tat,c2_tat,dc2_tat,c3_tat,dc3_tat,c4_tat,ocv2_tat,pg1_tat,pc1_tat,sa1_tat,socv1_tat,sa2_tat,socv2_tat,sa3_tat,socv3_tat,sa4_tat
0,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10747,양품,10,08열,33연,08단,03열,25연,03단,04열,64연,06단,3333,22열,08단,6801,3781,6451,6451,3417,22열,08단,268,6799,4108,544,22열,08단,266,539,6800,4146,625,625,332,22열,08단,265,6799,4112,544,22열,08단,266,539,135,4189,749,578,1190,171,307,22열,08단,266,6796,3674,7094,22열,08단,269,7088,136,3629,4282,4072,1091,210,2157,22열,08단,273,3764,22열,08단,2211,2114,3759,120,12열,01단,288,2086,3714,150,12열,01단,09열,42연,01단,3766.1,03열,64연,05단,3761.1,14열,37연,04단,3759.3,5,6.8,4718,37583,94,565,164,217066,86461,5732,119,3556,404,466,404,1797,3884,3428,118,290,290,53500,5,355357,5,215150,5,207532
1,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10748,양품,7,08열,33연,08단,03열,25연,03단,04열,64연,06단,3329,22열,08단,6797,3783,6390,6390,3385,22열,08단,268,6800,4103,545,22열,08단,266,540,6802,4144,657,657,349,22열,08단,265,6800,4110,544,22열,08단,266,539,136,4189,768,588,1286,180,313,22열,08단,266,6799,3674,7091,22열,08단,269,7085,136,3624,4268,4064,969,204,2153,22열,08단,273,3763,22열,08단,2224,2127,3758,120,12열,01단,288,2093,3714,150,12열,01단,09열,42연,01단,3765.2,03열,64연,05단,3760.2,14열,37연,04단,3758.4,5,6.8,4718,37574,94,565,73,217066,86461,5732,119,3556,404,466,404,1797,3884,3428,118,290,290,53500,5,355357,5,215150,5,207532
2,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10749,양품,19,08열,33연,08단,03열,25연,03단,04열,64연,06단,3329,22열,08단,6798,3781,6436,6436,3410,22열,08단,268,6799,4107,544,22열,08단,266,539,6801,4146,630,630,335,22열,08단,265,6799,4111,544,22열,08단,266,539,136,4189,753,579,1228,174,308,22열,08단,266,6799,3674,7079,22열,08단,269,7073,136,3630,4270,4058,1112,212,2150,22열,08단,273,3764,22열,08단,2217,2120,3759,120,12열,01단,288,2081,3715,150,12열,01단,09열,42연,01단,3765.9,03열,64연,05단,3760.8,14열,37연,04단,3759,5.1,6.9,4700,37579,91,585,-57,217066,86461,5732,118,3556,405,466,404,1797,3884,3428,118,290,290,53500,5,355357,5,215150,5,207532
3,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10750,양품,24,08열,33연,08단,03열,25연,03단,04열,64연,06단,3330,22열,08단,6799,3781,6474,6474,3430,22열,08단,268,6800,4108,545,22열,08단,266,540,6801,4146,627,627,333,22열,08단,265,6799,4111,544,22열,08단,266,539,136,4190,765,583,1328,182,310,22열,08단,266,6796,3675,7106,22열,08단,269,7100,136,3630,4304,4092,1129,212,2168,22열,08단,273,3764,22열,08단,2214,2117,3759,119,12열,01단,288,2083,3714,150,12열,01단,09열,42연,01단,3766.4,03열,64연,05단,3761.3,14열,37연,04단,3759.5,5.1,6.9,4688,37583,93,606,99,217066,86461,5732,118,3556,405,466,404,1797,3884,3428,118,290,290,53501,5,355357,5,215150,5,207532
4,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10751,양품,17,08열,33연,08단,03열,25연,03단,04열,64연,06단,3330,22열,08단,6799,3780,6486,6486,

In [166]:
df_raw.replace('.', np.NaN, inplace=True)
df_raw.replace(' ', np.NaN, inplace=True)

In [167]:
df_raw.dropna(inplace = True)

In [168]:
df_raw.shape

(43657, 130)

### **(21열, 22열)이랑 (6열, 9열, 18열) 나누기**

In [169]:
# 22열, 21열

df_raw22 = df_raw[df_raw['c4_box_col'] == '22열']
df_raw21 = df_raw[df_raw['c4_box_col'] == '21열']

df_raw_top = pd.concat([df_raw22, df_raw21], axis = 0)

print(df_raw_top.shape)
df_raw_top.head()

(4988, 130)


,lot_id,tray_id,dt_start,cell_id,judge,rta1_cell_no,rta1_box_col,rta1_box_row,rta1_box_dan,hta1_box_col,hta1_box_row,hta1_box_dan,rta2_box_col,rta2_box_row,rta2_box_dan,ocv1_ocv,ocv1_box_col,ocv1_box_dan,c1_curr_end,c1_voltage_avg,c1_capa,c1_ccval,c1_time_cc,c1_box_col,c1_box_dan,c1_temp_avg,dc1_curr_end,dc1_voltage_avg,dc1_capa,dc1_box_col,dc1_box_dan,dc1_temp_avg,dc1_capafit,c2_curr_end,c2_voltage_avg,c2_capa,c2_ccval,c2_time_cc,c2_box_col,c2_box_dan,c2_temp_avg,dc2_curr_end,dc2_voltage_avg,dc2_capa,dc2_box_col,dc2_box_dan,dc2_temp_avg,dc2_capafit,c3_curr_end,c3_voltage_avg,c3_capa,c3_ccval,c3_time_cv,c3_cvval,c3_time_cc,c3_box_col,c3_box_dan,c3_temp_avg,dc3_curr_end,dc3_voltage_avg,dc3_capa,dc3_box_col,dc3_box_dan,dc3_temp_avg,dc3_capafit,c4_curr_end,c4_voltage_avg,c4_capa,c4_ccval,c4_time_cv,c4_cvval,c4_time_cc,c4_box_col,c4_box_dan,c4_temp_avg,ocv2_ocv,ocv2_box_col,ocv2_box_dan,pg1_impfit,pg1_imp,pg1_voltage_start,pg1_curr_end,pg1_box_col,pg1_box_dan,pg1_temp_avg,pc1_imp,pc1_voltage_start,pc1_curr_end,pc1_box_col,pc1_box_dan,sa1_box_col,sa1_box_row,sa1_box_dan,socv1_ocv,sa2_box_col,sa2_box_row,sa2_box_dan,socv2_ocv,sa3_box_col,sa3_box_row,sa3_box_dan,socv3_ocv,ocv2_deltaocv,ocv1_deltaocv,m1_thick,m1_voltage,m1_res_ac,m1_mv,m1_voltage_an,rta1_tat,hta1_tat,rta2_tat,ocv1_tat,c1_tat,dc1_tat,c2_tat,dc2_tat,c3_tat,dc3_tat,c4_tat,ocv2_tat,pg1_tat,pc1_tat,sa1_tat,socv1_tat,sa2_tat,socv2_tat,sa3_tat,socv3_tat,sa4_tat
0,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10747,양품,10,08열,33연,08단,03열,25연,03단,04열,64연,06단,3333,22열,08단,6801,3781,6451,6451,3417,22열,08단,268,6799,4108,544,22열,08단,266,539,6800,4146,625,625,332,22열,08단,265,6799,4112,544,22열,08단,266,539,135,4189,749,578,1190,171,307,22열,08단,266,6796,3674,7094,22열,08단,269,7088,136,3629,4282,4072,1091,210,2157,22열,08단,273,3764,22열,08단,2211,2114,3759,120,12열,01단,288,2086,3714,150,12열,01단,09열,42연,01단,3766.1,03열,64연,05단,3761.1,14열,37연,04단,3759.3,5,6.8,4718,37583,94,565,164,217066,86461,5732,119,3556,404,466,404,1797,3884,3428,118,290,290,53500,5,355357,5,215150,5,207532
1,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10748,양품,7,08열,33연,08단,03열,25연,03단,04열,64연,06단,3329,22열,08단,6797,3783,6390,6390,3385,22열,08단,268,6800,4103,545,22열,08단,266,540,6802,4144,657,657,349,22열,08단,265,6800,4110,544,22열,08단,266,539,136,4189,768,588,1286,180,313,22열,08단,266,6799,3674,7091,22열,08단,269,7085,136,3624,4268,4064,969,204,2153,22열,08단,273,3763,22열,08단,2224,2127,3758,120,12열,01단,288,2093,3714,150,12열,01단,09열,42연,01단,3765.2,03열,64연,05단,3760.2,14열,37연,04단,3758.4,5,6.8,4718,37574,94,565,73,217066,86461,5732,119,3556,404,466,404,1797,3884,3428,118,290,290,53500,5,355357,5,215150,5,207532
2,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10749,양품,19,08열,33연,08단,03열,25연,03단,04열,64연,06단,3329,22열,08단,6798,3781,6436,6436,3410,22열,08단,268,6799,4107,544,22열,08단,266,539,6801,4146,630,630,335,22열,08단,265,6799,4111,544,22열,08단,266,539,136,4189,753,579,1228,174,308,22열,08단,266,6799,3674,7079,22열,08단,269,7073,136,3630,4270,4058,1112,212,2150,22열,08단,273,3764,22열,08단,2217,2120,3759,120,12열,01단,288,2081,3715,150,12열,01단,09열,42연,01단,3765.9,03열,64연,05단,3760.8,14열,37연,04단,3759,5.1,6.9,4700,37579,91,585,-57,217066,86461,5732,118,3556,405,466,404,1797,3884,3428,118,290,290,53500,5,355357,5,215150,5,207532
3,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10750,양품,24,08열,33연,08단,03열,25연,03단,04열,64연,06단,3330,22열,08단,6799,3781,6474,6474,3430,22열,08단,268,6800,4108,545,22열,08단,266,540,6801,4146,627,627,333,22열,08단,265,6799,4111,544,22열,08단,266,539,136,4190,765,583,1328,182,310,22열,08단,266,6796,3675,7106,22열,08단,269,7100,136,3630,4304,4092,1129,212,2168,22열,08단,273,3764,22열,08단,2214,2117,3759,119,12열,01단,288,2083,3714,150,12열,01단,09열,42연,01단,3766.4,03열,64연,05단,3761.3,14열,37연,04단,3759.5,5.1,6.9,4688,37583,93,606,99,217066,86461,5732,118,3556,405,466,404,1797,3884,3428,118,290,290,53501,5,355357,5,215150,5,207532
4,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10751,양품,17,08열,33연,08단,03열,25연,03단,04열,64연,06단,3330,22열,08단,6799,3780,6486,6486,

In [170]:
# 9열, 4열, 6열

df_raw9 = df_raw[df_raw['c4_box_col'] == '09열']
df_raw4 = df_raw[df_raw['c4_box_col'] == '04열']
df_raw6 = df_raw[df_raw['c4_box_col'] == '06열']

df_raw_bot = pd.concat([df_raw9, df_raw4], axis = 0)
df_raw_bot = pd.concat([df_raw_bot, df_raw6], axis = 0)

print(df_raw_bot.shape)
df_raw_bot.head()

(4876, 130)


,lot_id,tray_id,dt_start,cell_id,judge,rta1_cell_no,rta1_box_col,rta1_box_row,rta1_box_dan,hta1_box_col,hta1_box_row,hta1_box_dan,rta2_box_col,rta2_box_row,rta2_box_dan,ocv1_ocv,ocv1_box_col,ocv1_box_dan,c1_curr_end,c1_voltage_avg,c1_capa,c1_ccval,c1_time_cc,c1_box_col,c1_box_dan,c1_temp_avg,dc1_curr_end,dc1_voltage_avg,dc1_capa,dc1_box_col,dc1_box_dan,dc1_temp_avg,dc1_capafit,c2_curr_end,c2_voltage_avg,c2_capa,c2_ccval,c2_time_cc,c2_box_col,c2_box_dan,c2_temp_avg,dc2_curr_end,dc2_voltage_avg,dc2_capa,dc2_box_col,dc2_box_dan,dc2_temp_avg,dc2_capafit,c3_curr_end,c3_voltage_avg,c3_capa,c3_ccval,c3_time_cv,c3_cvval,c3_time_cc,c3_box_col,c3_box_dan,c3_temp_avg,dc3_curr_end,dc3_voltage_avg,dc3_capa,dc3_box_col,dc3_box_dan,dc3_temp_avg,dc3_capafit,c4_curr_end,c4_voltage_avg,c4_capa,c4_ccval,c4_time_cv,c4_cvval,c4_time_cc,c4_box_col,c4_box_dan,c4_temp_avg,ocv2_ocv,ocv2_box_col,ocv2_box_dan,pg1_impfit,pg1_imp,pg1_voltage_start,pg1_curr_end,pg1_box_col,pg1_box_dan,pg1_temp_avg,pc1_imp,pc1_voltage_start,pc1_curr_end,pc1_box_col,pc1_box_dan,sa1_box_col,sa1_box_row,sa1_box_dan,socv1_ocv,sa2_box_col,sa2_box_row,sa2_box_dan,socv2_ocv,sa3_box_col,sa3_box_row,sa3_box_dan,socv3_ocv,ocv2_deltaocv,ocv1_deltaocv,m1_thick,m1_voltage,m1_res_ac,m1_mv,m1_voltage_an,rta1_tat,hta1_tat,rta2_tat,ocv1_tat,c1_tat,dc1_tat,c2_tat,dc2_tat,c3_tat,dc3_tat,c4_tat,ocv2_tat,pg1_tat,pc1_tat,sa1_tat,socv1_tat,sa2_tat,socv2_tat,sa3_tat,socv3_tat,sa4_tat
396,LOT-100,TRAY-1008,12-01-2021 오전 6:36:25,CELL-10175,양품,1,07열,05연,01단,01열,24연,06단,06열,09연,08단,3339,09열,07단,6801,3782,6469,6469,3426,09열,07단,280,6800,4109,545,09열,07단,279,536,6800,4147,627,627,333,09열,07단,278,6799,4113,544,09열,07단,277,536,136,4188,737,578,978,158,307,09열,07단,275,6797,3675,7127,09열,07단,286,7116,135,3618,4307,4113,859,194,2179,09열,07단,293,3763,09열,07단,2218,2070,3761,120,06열,01단,290,2052,3715,150,06열,01단,08열,37연,12단,3766.2,14열,44연,02단,3761.1,10열,48연,05단,3759.3,5.1,6.9,4725,37583,92,566,144,217901,86480,4978,118,3556,405,465,405,1748,3897,3411,118,290,290,54522,5,355271,5,215550,5,208516
397,LOT-100,TRAY-1008,12-01-2021 오전 6:36:25,CELL-10176,양품,7,07열,05연,01단,01열,24연,06단,06열,09연,08단,3338,09열,07단,6798,3782,6448,6448,3416,09열,07단,280,6799,4107,544,09열,07단,279,535,6801,4145,635,635,337,09열,07단,278,6799,4112,544,09열,07단,277,536,135,4190,760,581,1304,179,309,09열,07단,275,6799,3674,7108,09열,07단,286,7097,135,3631,4299,4085,1132,214,2164,09열,07단,293,3765,09열,07단,2249,2101,3761,119,06열,01단,290,2083,3715,150,06열,01단,08열,37연,12단,3766.7,14열,44연,02단,3761.6,10열,48연,05단,3759.8,5.1,6.9,4733,37588,95,567,130,217901,86480,4978,118,3556,405,465,405,1748,3897,3411,118,290,290,54522,5,355271,5,215550,5,208516
398,LOT-100,TRAY-1008,12-01-2021 오전 6:36:25,CELL-10177,양품,24,07열,05연,01단,01열,24연,06단,06열,09연,08단,3337,09열,07단,6798,3782,6445,6445,3415,09열,07단,280,6799,4107,544,09열,07단,279,535,6801,4146,630,630,335,09열,07단,278,6799,4112,544,09열,07단,277,536,136,4189,749,579,1186,169,308,09열,07단,275,6799,3674,7104,09열,07단,286,7093,136,3628,4286,4079,1057,208,2161,09열,07단,293,3764,09열,07단,2247,2099,3760,119,06열,01단,290,2088,3715,150,06열,01단,08열,37연,12단,3766.3,14열,44연,02단,3761.2,10열,48연,05단,3759.5,5.1,6.8,4715,37583,92,587,-92,217901,86480,4978,118,3556,405,465,405,1748,3897,3411,118,290,290,54522,5,355271,5,215550,5,208516
399,LOT-100,TRAY-1008,12-01-2021 오전 6:36:25,CELL-10178,양품,5,07열,05연,01단,01열,24연,06단,06열,09연,08단,3337,09열,07단,6802,3782,6455,6455,3419,09열,07단,280,6801,4108,545,09열,07단,279,536,6796,4146,636,636,338,09열,07단,278,6799,4112,544,09열,07단,277,536,136,4189,747,581,1110,166,309,09열,07단,275,6798,3675,7108,09열,07단,286,7097,136,3627,4300,4095,1034,206,2169,09열,07단,293,3764,09열,07단,2230,2082,3761,119,06열,01단,290,2064,3716,150,06열,01단,08열,37연,12단,3766.8,14열,44연,02단,3761.6,10열,48연,05단,3759.9,5.2,6.9,4742,37588,94,567,56,217901,86480,4978,118,3556,405,465,405,1748,3897,3411,118,290,290,54522,5,355271,5,215550,5,208516
400,LOT-100,TRAY-1008,12-01-2021 오전 6:36:25,CELL-10179,양품,14,07열,05연,01단,01열,24연,06단,06열,09연,08단,3335,09열,07단,6802,3781,6

In [171]:
df_raw_total = pd.concat([df_raw_top, df_raw_bot], axis = 0)
df_raw_total = df_raw_total.reset_index()

print(df_raw_total.shape)
df_raw_total.tail()

(9864, 131)


,index,lot_id,tray_id,dt_start,cell_id,judge,rta1_cell_no,rta1_box_col,rta1_box_row,rta1_box_dan,hta1_box_col,hta1_box_row,hta1_box_dan,rta2_box_col,rta2_box_row,rta2_box_dan,ocv1_ocv,ocv1_box_col,ocv1_box_dan,c1_curr_end,c1_voltage_avg,c1_capa,c1_ccval,c1_time_cc,c1_box_col,c1_box_dan,c1_temp_avg,dc1_curr_end,dc1_voltage_avg,dc1_capa,dc1_box_col,dc1_box_dan,dc1_temp_avg,dc1_capafit,c2_curr_end,c2_voltage_avg,c2_capa,c2_ccval,c2_time_cc,c2_box_col,c2_box_dan,c2_temp_avg,dc2_curr_end,dc2_voltage_avg,dc2_capa,dc2_box_col,dc2_box_dan,dc2_temp_avg,dc2_capafit,c3_curr_end,c3_voltage_avg,c3_capa,c3_ccval,c3_time_cv,c3_cvval,c3_time_cc,c3_box_col,c3_box_dan,c3_temp_avg,dc3_curr_end,dc3_voltage_avg,dc3_capa,dc3_box_col,dc3_box_dan,dc3_temp_avg,dc3_capafit,c4_curr_end,c4_voltage_avg,c4_capa,c4_ccval,c4_time_cv,c4_cvval,c4_time_cc,c4_box_col,c4_box_dan,c4_temp_avg,ocv2_ocv,ocv2_box_col,ocv2_box_dan,pg1_impfit,pg1_imp,pg1_voltage_start,pg1_curr_end,pg1_box_col,pg1_box_dan,pg1_temp_avg,pc1_imp,pc1_voltage_start,pc1_curr_end,pc1_box_col,pc1_box_dan,sa1_box_col,sa1_box_row,sa1_box_dan,socv1_ocv,sa2_box_col,sa2_box_row,sa2_box_dan,socv2_ocv,sa3_box_col,sa3_box_row,sa3_box_dan,socv3_ocv,ocv2_deltaocv,ocv1_deltaocv,m1_thick,m1_voltage,m1_res_ac,m1_mv,m1_voltage_an,rta1_tat,hta1_tat,rta2_tat,ocv1_tat,c1_tat,dc1_tat,c2_tat,dc2_tat,c3_tat,dc3_tat,c4_tat,ocv2_tat,pg1_tat,pc1_tat,sa1_tat,socv1_tat,sa2_tat,socv2_tat,sa3_tat,socv3_tat,sa4_tat
9859,43428,LOT-144,TRAY-2773,12-17-2021 오후 2:36:45,CELL-53553,양품,12,07열,58연,09단,01열,22연,10단,07열,42연,04단,3350,06열,11단,6798,3781,6493,6493,3440,06열,11단,287,6800,4113,544,06열,11단,290,532,6800,4148,601,601,319,06열,11단,290,6800,4116,544,06열,11단,290,532,136,4188,720,572,964,148,304,06열,11단,288,6796,3676,7133,06열,11단,287,7122,136,3623,4308,4111,949,197,2178,06열,11단,288,3763,06열,11단,2236,2110,3760,120,08열,01단,275,2101,3715,150,08열,01단,12열,27연,04단,3766.0,13열,57연,05단,3761.0,10열,16연,13단,3759.3,5.0,6.7,4766,37586,94,542,-38,155401,87263,155952,119,3565,405,449,405,1605,3893,3370,118,290,291,42934,5,345063,5,217121,5,158024
9860,43429,LOT-144,TRAY-2773,12-17-2021 오후 2:36:45,CELL-53554,양품,9,07열,58연,09단,01열,22연,10단,07열,42연,04단,3352,06열,11단,6798,3780,6486,6486,3436,06열,11단,287,6800,4114,544,06열,11단,290,532,6801,4148,603,603,321,06열,11단,290,6800,4116,544,06열,11단,290,532,136,4189,726,575,1051,152,306,06열,11단,288,6795,3677,7129,06열,11단,287,7118,136,3626,4314,4115,1024,199,2180,06열,11단,288,3764,06열,11단,2237,2111,3761,120,08열,01단,275,2114,3716,150,08열,01단,12열,27연,04단,3766.6,13열,57연,05단,3761.7,10열,16연,13단,3760.0,4.9,6.6,4770,37593,94,542,-65,155401,87263,155952,119,3565,405,449,405,1605,3893,3370,118,290,291,42934,5,345063,5,217121,5,158024
9861,43430,LOT-144,TRAY-2773,12-17-2021 오후 2:36:45,CELL-53555,양품,24,07열,58연,09단,01열,22연,10단,07열,42연,04단,3350,06열,11단,6800,3780,6506,6506,3447,06열,11단,287,6800,4114,545,06열,11단,290,533,6799,4149,595,595,316,06열,11단,290,6800,4116,545,06열,11단,290,533,135,4188,715,569,947,146,303,06열,11단,288,6798,3677,7116,06열,11단,287,7105,135,3622,4315,4119,945,196,2182,06열,11단,288,3763,06열,11단,2231,2105,3761,119,08열,01단,275,2090,3716,150,08열,01단,12열,27연,04단,3766.2,13열,57연,05단,3761.2,10열,16연,13단,3759.5,5.0,6.7,4734,37586,91,587,121,155401,87263,155952,119,3565,404,449,405,1605,3893,3369,118,290,291,42934,5,345063,5,217121,5,158024
9862,43431,LOT-144,TRAY-2773,12-17-2021 오후 2:36:45,CELL-54255,양품,1,07열,58연,09단,01열,22연,10단,07열,42연,04단,3325,06열,11단,6799,3780,6453,6453,3418,06열,11단,287,6800,4109,544,06열,11단,290,532,6801,4146,629,629,334,06열,11단,290,6800,4114,545,06열,11단,290,533,135,4189,740,577,1161,163,307,06열,11단,288,6796,3675,7084,06열,11단,287,7073,135,3624,4266,4064,974,202,2153,06열,11단,288,3763,06열,11단,2249,2123,3760,120,08열,01단,275,2129,3716,150,08열,01단,12열,27연,04단,3765.8,13열,57연,05단,3760.7,10열,16연,13단,3759.0,5.1,6.8,4784,37582,92,564,-91,155401,87263,155952,119,3565,405,449,405,1605,3893,3370,118,290,290,42934,6,345063,5,217121,5,158024
9863,43432,LOT-144,TRAY-2773,12-17-2021 오후 2:36:45,CELL-54256,양품,2,07열,58연,09단,0

In [172]:
df_raw_total.loc[df_raw_total['c4_box_col'] == '04열', 'c4_box_col'] = 0
df_raw_total.loc[df_raw_total['c4_box_col'] == '06열', 'c4_box_col'] = 0
df_raw_total.loc[df_raw_total['c4_box_col'] == '09열', 'c4_box_col'] = 0

df_raw_total.loc[df_raw_total['c4_box_col'] == '21열', 'c4_box_col'] = 1
df_raw_total.loc[df_raw_total['c4_box_col'] == '22열', 'c4_box_col'] = 1

In [173]:
df_raw_total['c4_box_col'].value_counts()

1    4988
0    4876
Name: c4_box_col, dtype: int64

In [174]:
df_raw_total['c4_box_col'] = df_raw_total['c4_box_col'].astype(int)

#### **카이제곱검정**
- 설비와 불량 여부 사이에 관련성이 있는지 검정


In [ ]:
ct = pd.crosstab(df_raw_total['c4_box_col'], df_raw_total['judge'])
ct

judge,불량,양품
c4_box_col,,
0,46,4830
1,421,4567


In [ ]:
stat, pval, df, expected = stats.chi2_contingency(observed = ct)

print('***** 카이제곱검정 *****')
print('H0 : 설비에 따라 양품불량개수의 차이가 없다.')
print('H1 : 설비에 따라 양품불량개수의 차이가 있다.')
print('')
print('통계량 : ', stat.round(3))
print('pvalue : ', pval.round(3))

***** 카이제곱검정 *****
H0 : 설비에 따라 양품불량개수의 차이가 없다.
H1 : 설비에 따라 양품불량개수의 차이가 있다.

통계량 :  305.593
pvalue :  0.0


- 검정통계량 305.593에 대한 유의확률 = 0.0 < 0.05이므로 귀무가설을 기각한다.
- 즉, 설비에 따라 양품불량개수의 차이가 있다.

### **ANOVA 검정**


In [ ]:
# ANOVA 실행

print('***** ANOVA분석 *****')
print('H0 : 설비에 따라 온도에 차이가 없다.')
print('H1 : 설비에 따라 온도에 차이가 있다.')
print('')
model = ols('c4_temp_avg ~ C(c4_box_col)', df_raw_total).fit()
anova_lm(model)

***** ANOVA분석 *****
H0 : 설비에 따라 온도에 차이가 없다.
H1 : 설비에 따라 온도에 차이가 있다.



,df,sum_sq,mean_sq,F,PR(>F)
C(c4_box_col),1.0,7.271406e+05,727140.612131,3634.647688,0.0
Residual,9862.0,1.972973e+06,200.058073,NaN,NaN


- 검정통계량 F = 3634.647688에 대한 유의확률 0.0 < 0.05이므로 귀무가설을 기각한다.
- 설비에 따라 온도에 차이가 있다.

### **2 sample t-test**



In [ ]:
# 정규성 검정

statistic1, p1 = stats.shapiro(df_raw_total.loc[df_raw_total['judge'] == '불량', 'c4_temp_avg'])
print("----------- 불량에 대한 정규성 검정 -----------")
print('H0 : 정규성을 만족한다.')
print('H1 : 정규성을 만족하지 않는다.')
print('')
print('통계량 : ', statistic1)
print('pvalue : ', p1)
print('')

statistic2, p2 = stats.shapiro(df_raw_total.loc[df_raw_total['judge'] == '양품', 'c4_temp_avg'])
print("----------- 양품에 대한 정규성 검정 -----------")
print('H0 : 정규성을 만족한다.')
print('H1 : 정규성을 만족하지 않는다.')
print('')
print('통계량 : ', statistic2)
print('pvalue : ', p2)
print('')

----------- 불량에 대한 정규성 검정 -----------
H0 : 정규성을 만족한다.
H1 : 정규성을 만족하지 않는다.

통계량 :  0.8832783102989197
pvalue :  2.611841188904814e-18

----------- 양품에 대한 정규성 검정 -----------
H0 : 정규성을 만족한다.
H1 : 정규성을 만족하지 않는다.

통계량 :  0.9911037683486938
pvalue :  8.634443905938565e-24



/usr/local/lib/python3.7/dist-packages/scipy/stats/morestats.py:1760: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


- 불량과 양품 개수에 따른 온도의 유의확률 0.000, 0.000 < 0.05이므로 귀무가설을 기각한다.
- 두 집단 모두 정규성을 만족하지 않으므로 맨휘트니 검정을 통해 t검정을 진행한다.

In [177]:
big = df_raw_total[df_raw_total['c4_box_col'] == 1]
big.head()

,index,lot_id,tray_id,dt_start,cell_id,judge,rta1_cell_no,rta1_box_col,rta1_box_row,rta1_box_dan,hta1_box_col,hta1_box_row,hta1_box_dan,rta2_box_col,rta2_box_row,rta2_box_dan,ocv1_ocv,ocv1_box_col,ocv1_box_dan,c1_curr_end,c1_voltage_avg,c1_capa,c1_ccval,c1_time_cc,c1_box_col,c1_box_dan,c1_temp_avg,dc1_curr_end,dc1_voltage_avg,dc1_capa,dc1_box_col,dc1_box_dan,dc1_temp_avg,dc1_capafit,c2_curr_end,c2_voltage_avg,c2_capa,c2_ccval,c2_time_cc,c2_box_col,c2_box_dan,c2_temp_avg,dc2_curr_end,dc2_voltage_avg,dc2_capa,dc2_box_col,dc2_box_dan,dc2_temp_avg,dc2_capafit,c3_curr_end,c3_voltage_avg,c3_capa,c3_ccval,c3_time_cv,c3_cvval,c3_time_cc,c3_box_col,c3_box_dan,c3_temp_avg,dc3_curr_end,dc3_voltage_avg,dc3_capa,dc3_box_col,dc3_box_dan,dc3_temp_avg,dc3_capafit,c4_curr_end,c4_voltage_avg,c4_capa,c4_ccval,c4_time_cv,c4_cvval,c4_time_cc,c4_box_col,c4_box_dan,c4_temp_avg,ocv2_ocv,ocv2_box_col,ocv2_box_dan,pg1_impfit,pg1_imp,pg1_voltage_start,pg1_curr_end,pg1_box_col,pg1_box_dan,pg1_temp_avg,pc1_imp,pc1_voltage_start,pc1_curr_end,pc1_box_col,pc1_box_dan,sa1_box_col,sa1_box_row,sa1_box_dan,socv1_ocv,sa2_box_col,sa2_box_row,sa2_box_dan,socv2_ocv,sa3_box_col,sa3_box_row,sa3_box_dan,socv3_ocv,ocv2_deltaocv,ocv1_deltaocv,m1_thick,m1_voltage,m1_res_ac,m1_mv,m1_voltage_an,rta1_tat,hta1_tat,rta2_tat,ocv1_tat,c1_tat,dc1_tat,c2_tat,dc2_tat,c3_tat,dc3_tat,c4_tat,ocv2_tat,pg1_tat,pc1_tat,sa1_tat,socv1_tat,sa2_tat,socv2_tat,sa3_tat,socv3_tat,sa4_tat
0,0,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10747,양품,10,08열,33연,08단,03열,25연,03단,04열,64연,06단,3333,22열,08단,6801,3781,6451,6451,3417,22열,08단,268,6799,4108,544,22열,08단,266,539,6800,4146,625,625,332,22열,08단,265,6799,4112,544,22열,08단,266,539,135,4189,749,578,1190,171,307,22열,08단,266,6796,3674,7094,22열,08단,269,7088,136,3629,4282,4072,1091,210,2157,1,08단,273,3764,22열,08단,2211,2114,3759,120,12열,01단,288,2086,3714,150,12열,01단,09열,42연,01단,3766.1,03열,64연,05단,3761.1,14열,37연,04단,3759.3,5,6.8,4718,37583,94,565,164,217066,86461,5732,119,3556,404,466,404,1797,3884,3428,118,290,290,53500,5,355357,5,215150,5,207532
1,1,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10748,양품,7,08열,33연,08단,03열,25연,03단,04열,64연,06단,3329,22열,08단,6797,3783,6390,6390,3385,22열,08단,268,6800,4103,545,22열,08단,266,540,6802,4144,657,657,349,22열,08단,265,6800,4110,544,22열,08단,266,539,136,4189,768,588,1286,180,313,22열,08단,266,6799,3674,7091,22열,08단,269,7085,136,3624,4268,4064,969,204,2153,1,08단,273,3763,22열,08단,2224,2127,3758,120,12열,01단,288,2093,3714,150,12열,01단,09열,42연,01단,3765.2,03열,64연,05단,3760.2,14열,37연,04단,3758.4,5,6.8,4718,37574,94,565,73,217066,86461,5732,119,3556,404,466,404,1797,3884,3428,118,290,290,53500,5,355357,5,215150,5,207532
2,2,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10749,양품,19,08열,33연,08단,03열,25연,03단,04열,64연,06단,3329,22열,08단,6798,3781,6436,6436,3410,22열,08단,268,6799,4107,544,22열,08단,266,539,6801,4146,630,630,335,22열,08단,265,6799,4111,544,22열,08단,266,539,136,4189,753,579,1228,174,308,22열,08단,266,6799,3674,7079,22열,08단,269,7073,136,3630,4270,4058,1112,212,2150,1,08단,273,3764,22열,08단,2217,2120,3759,120,12열,01단,288,2081,3715,150,12열,01단,09열,42연,01단,3765.9,03열,64연,05단,3760.8,14열,37연,04단,3759,5.1,6.9,4700,37579,91,585,-57,217066,86461,5732,118,3556,405,466,404,1797,3884,3428,118,290,290,53500,5,355357,5,215150,5,207532
3,3,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10750,양품,24,08열,33연,08단,03열,25연,03단,04열,64연,06단,3330,22열,08단,6799,3781,6474,6474,3430,22열,08단,268,6800,4108,545,22열,08단,266,540,6801,4146,627,627,333,22열,08단,265,6799,4111,544,22열,08단,266,539,136,4190,765,583,1328,182,310,22열,08단,266,6796,3675,7106,22열,08단,269,7100,136,3630,4304,4092,1129,212,2168,1,08단,273,3764,22열,08단,2214,2117,3759,119,12열,01단,288,2083,3714,150,12열,01단,09열,42연,01단,3766.4,03열,64연,05단,3761.3,14열,37연,04단,3759.5,5.1,6.9,4688,37583,93,606,99,217066,86461,5732,118,3556,405,466,404,1797,3884,3428,118,290,290,53501,5,355357,5,215150,5,207532
4,4,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10751,양품,17,08열,33연,08단,03열,25연,03단,04열,64연,06단,3330,22열,08단,6799,3780,64

In [178]:
small = df_raw_total[df_raw_total['c4_box_col'] == 0]
small.head()

,index,lot_id,tray_id,dt_start,cell_id,judge,rta1_cell_no,rta1_box_col,rta1_box_row,rta1_box_dan,hta1_box_col,hta1_box_row,hta1_box_dan,rta2_box_col,rta2_box_row,rta2_box_dan,ocv1_ocv,ocv1_box_col,ocv1_box_dan,c1_curr_end,c1_voltage_avg,c1_capa,c1_ccval,c1_time_cc,c1_box_col,c1_box_dan,c1_temp_avg,dc1_curr_end,dc1_voltage_avg,dc1_capa,dc1_box_col,dc1_box_dan,dc1_temp_avg,dc1_capafit,c2_curr_end,c2_voltage_avg,c2_capa,c2_ccval,c2_time_cc,c2_box_col,c2_box_dan,c2_temp_avg,dc2_curr_end,dc2_voltage_avg,dc2_capa,dc2_box_col,dc2_box_dan,dc2_temp_avg,dc2_capafit,c3_curr_end,c3_voltage_avg,c3_capa,c3_ccval,c3_time_cv,c3_cvval,c3_time_cc,c3_box_col,c3_box_dan,c3_temp_avg,dc3_curr_end,dc3_voltage_avg,dc3_capa,dc3_box_col,dc3_box_dan,dc3_temp_avg,dc3_capafit,c4_curr_end,c4_voltage_avg,c4_capa,c4_ccval,c4_time_cv,c4_cvval,c4_time_cc,c4_box_col,c4_box_dan,c4_temp_avg,ocv2_ocv,ocv2_box_col,ocv2_box_dan,pg1_impfit,pg1_imp,pg1_voltage_start,pg1_curr_end,pg1_box_col,pg1_box_dan,pg1_temp_avg,pc1_imp,pc1_voltage_start,pc1_curr_end,pc1_box_col,pc1_box_dan,sa1_box_col,sa1_box_row,sa1_box_dan,socv1_ocv,sa2_box_col,sa2_box_row,sa2_box_dan,socv2_ocv,sa3_box_col,sa3_box_row,sa3_box_dan,socv3_ocv,ocv2_deltaocv,ocv1_deltaocv,m1_thick,m1_voltage,m1_res_ac,m1_mv,m1_voltage_an,rta1_tat,hta1_tat,rta2_tat,ocv1_tat,c1_tat,dc1_tat,c2_tat,dc2_tat,c3_tat,dc3_tat,c4_tat,ocv2_tat,pg1_tat,pc1_tat,sa1_tat,socv1_tat,sa2_tat,socv2_tat,sa3_tat,socv3_tat,sa4_tat
4988,396,LOT-100,TRAY-1008,12-01-2021 오전 6:36:25,CELL-10175,양품,1,07열,05연,01단,01열,24연,06단,06열,09연,08단,3339,09열,07단,6801,3782,6469,6469,3426,09열,07단,280,6800,4109,545,09열,07단,279,536,6800,4147,627,627,333,09열,07단,278,6799,4113,544,09열,07단,277,536,136,4188,737,578,978,158,307,09열,07단,275,6797,3675,7127,09열,07단,286,7116,135,3618,4307,4113,859,194,2179,0,07단,293,3763,09열,07단,2218,2070,3761,120,06열,01단,290,2052,3715,150,06열,01단,08열,37연,12단,3766.2,14열,44연,02단,3761.1,10열,48연,05단,3759.3,5.1,6.9,4725,37583,92,566,144,217901,86480,4978,118,3556,405,465,405,1748,3897,3411,118,290,290,54522,5,355271,5,215550,5,208516
4989,397,LOT-100,TRAY-1008,12-01-2021 오전 6:36:25,CELL-10176,양품,7,07열,05연,01단,01열,24연,06단,06열,09연,08단,3338,09열,07단,6798,3782,6448,6448,3416,09열,07단,280,6799,4107,544,09열,07단,279,535,6801,4145,635,635,337,09열,07단,278,6799,4112,544,09열,07단,277,536,135,4190,760,581,1304,179,309,09열,07단,275,6799,3674,7108,09열,07단,286,7097,135,3631,4299,4085,1132,214,2164,0,07단,293,3765,09열,07단,2249,2101,3761,119,06열,01단,290,2083,3715,150,06열,01단,08열,37연,12단,3766.7,14열,44연,02단,3761.6,10열,48연,05단,3759.8,5.1,6.9,4733,37588,95,567,130,217901,86480,4978,118,3556,405,465,405,1748,3897,3411,118,290,290,54522,5,355271,5,215550,5,208516
4990,398,LOT-100,TRAY-1008,12-01-2021 오전 6:36:25,CELL-10177,양품,24,07열,05연,01단,01열,24연,06단,06열,09연,08단,3337,09열,07단,6798,3782,6445,6445,3415,09열,07단,280,6799,4107,544,09열,07단,279,535,6801,4146,630,630,335,09열,07단,278,6799,4112,544,09열,07단,277,536,136,4189,749,579,1186,169,308,09열,07단,275,6799,3674,7104,09열,07단,286,7093,136,3628,4286,4079,1057,208,2161,0,07단,293,3764,09열,07단,2247,2099,3760,119,06열,01단,290,2088,3715,150,06열,01단,08열,37연,12단,3766.3,14열,44연,02단,3761.2,10열,48연,05단,3759.5,5.1,6.8,4715,37583,92,587,-92,217901,86480,4978,118,3556,405,465,405,1748,3897,3411,118,290,290,54522,5,355271,5,215550,5,208516
4991,399,LOT-100,TRAY-1008,12-01-2021 오전 6:36:25,CELL-10178,양품,5,07열,05연,01단,01열,24연,06단,06열,09연,08단,3337,09열,07단,6802,3782,6455,6455,3419,09열,07단,280,6801,4108,545,09열,07단,279,536,6796,4146,636,636,338,09열,07단,278,6799,4112,544,09열,07단,277,536,136,4189,747,581,1110,166,309,09열,07단,275,6798,3675,7108,09열,07단,286,7097,136,3627,4300,4095,1034,206,2169,0,07단,293,3764,09열,07단,2230,2082,3761,119,06열,01단,290,2064,3716,150,06열,01단,08열,37연,12단,3766.8,14열,44연,02단,3761.6,10열,48연,05단,3759.9,5.2,6.9,4742,37588,94,567,56,217901,86480,4978,118,3556,405,465,405,1748,3897,3411,118,290,290,54522,5,355271,5,215550,5,208516
4992,400,LOT-100,TRAY-1008,12-01-2021 오전 6:36:25,CELL-10179,양품,14,07열,05연,01단,01열,24연,06단,06열,09연,08단,3

In [181]:
# 맨휘트니 검정 (비모수적 t검정)
# 불량 여부에 따라 온도에 차이가 있는지

stat, p = stats.mannwhitneyu(big['c4_voltage_avg'], small['c4_voltage_avg'])

print('stat : {0:0.5f}'.format(stat))
print('p-value : {0:0.5f}'.format(p))

stat : 14539111.50000
p-value : 0.00000


- 불량 개수가 많은 집단과 적은 집단의 온도 차이에 대한 유의확률 0.000 < 0.05이므로 귀무가설을 기각한다.
- 양품과 불량 사이에 온도 차이가 존재한다.

In [ ]:
# 맨휘트니 검정 (비모수적 t검정)
# 불량 여부에 따라 전압에 차이가 있는지

stat, p = stats.mannwhitneyu(big['c4_temp_avg'], small['c4_temp_avg'])

print('stat : {0:0.5f}'.format(stat))
print('p-value : {0:0.5f}'.format(p))

- 불량 개수가 많은 집단과 적은 집단의 전압 차이에 대한 유의확률 0.000 < 0.05이므로 귀무가설을 기각한다.
- 양품과 불량 사이에 전압 차이가 존재한다.

In [182]:
# 맨휘트니 검정 (비모수적 t검정)
# 불량 여부에 따라 전류에 차이가 있는지

stat, p = stats.mannwhitneyu(big['c4_curr_end'], small['c4_curr_end'])

print('stat : {0:0.5f}'.format(stat))
print('p-value : {0:0.5f}'.format(p))

stat : 12396916.00000
p-value : 0.02905


- 불량 개수가 많은 집단과 적은 집단의 전류 차이에 대한 유의확률 0.029 < 0.05이므로 귀무가설을 기각한다.
- 양품과 불량 사이에 전류 차이가 존재한다.

**가설검정 결론**

- 중요한 공정조건 : 온도, 전압, 전류